In [80]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,RepeatedStratifiedKFold,cross_val_score
from sklearn.preprocessing import StandardScaler
from numpy import mean,std
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix,recall_score
import warnings
warnings.filterwarnings('ignore')
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout

## Loading Data

In [30]:
data = data = pd.read_csv('../Datasets/telecommunications_churn.csv',sep=';')
data.head()

,account_length,voice_mail_plan,voice_mail_messages,day_mins,evening_mins,night_mins,international_mins,customer_service_calls,international_plan,day_calls,day_charge,evening_calls,evening_charge,night_calls,night_charge,international_calls,international_charge,total_charge,churn
0,128,1,25,265.1,197.4,244.7,10.0,1,0,110,45.07,99,16.78,91,11.01,3,2.70,75.56,0
1,107,1,26,161.6,195.5,254.4,13.7,1,0,123,27.47,103,16.62,103,11.45,3,3.70,59.24,0
2,137,0,0,243.4,121.2,162.6,12.2,0,0,114,41.38,110,10.30,104,7.32,5,3.29,62.29,0
3,84,0,0,299.4,61.9,196.9,6.6,2,1,71,50.90,88,5.26,89,8.86,7,1.78,66.80,0
4,75,0,0,166.7,148.3,186.9,10.1,3,1,113,28.34,122,12.61,121,8.41,3,2.73,52.09,0


## Scaling Data and Seperating X and y

In [31]:
std_scr = StandardScaler()
X = data.drop(labels='churn',axis=1)
cols = X.columns
X = pd.DataFrame(std_scr.fit_transform(X))
X.columns = cols
y = data[['churn']]
X.shape,y.shape

((3333, 18), (3333, 1))

## Train Test Split

In [32]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,shuffle=True,random_state=12,stratify=y)

### Dict to record model performance

In [34]:

my_dict = {'model':[], 'accuracy': [],'precision':[],'recall':[]}

### Function to pridict and record model performance

In [35]:
def populate_matrix(fittedModel):
    y_pred = fittedModel.predict(x_test)
    accuracy = round(accuracy_score(y_test,y_pred)*100,2)
    precision = round(precision_score(y_test,y_pred)*100,2)
    recall = round(recall_score(y_test,y_pred)*100,2)
    model = str(fittedModel)
    my_dict["model"].append(model)
    my_dict["accuracy"].append(accuracy)
    my_dict["precision"].append(precision)
    my_dict["recall"].append(recall)


## Model Validation

## Logistic Model

In [36]:
# evaluate model
logistic = LogisticRegression()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(estimator=logistic,X=X,y=y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))


Accuracy: 0.861 (0.009)


### Model Training

In [37]:

logistic.fit(x_train,y_train)
populate_matrix(logistic)
print(my_dict)

{'model': ['LogisticRegression()'], 'accuracy': [86.51], 'precision': [56.6], 'recall': [30.93]}


## Decision Tree

In [38]:
dtree = DecisionTreeClassifier()
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(estimator=dtree,X=X,y=y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.948 (0.010)


### Model Training

In [39]:
dtree.fit(x_train,y_train)
populate_matrix(dtree)
print(my_dict)

{'model': ['LogisticRegression()', 'DecisionTreeClassifier()'], 'accuracy': [86.51, 93.4], 'precision': [56.6, 73.45], 'recall': [30.93, 85.57]}


## RandomForest

In [40]:
rf = RandomForestClassifier()
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(estimator=rf,X=X,y=y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.978 (0.008)


### Model Training

In [41]:
rf.fit(x_train,y_train)
populate_matrix(rf)
print(my_dict)

{'model': ['LogisticRegression()', 'DecisionTreeClassifier()', 'RandomForestClassifier()'], 'accuracy': [86.51, 93.4, 97.45], 'precision': [56.6, 73.45, 100.0], 'recall': [30.93, 85.57, 82.47]}


## Gradient Boosting

In [42]:

gboost = GradientBoostingClassifier()
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(estimator=gboost,X=X,y=y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.978 (0.007)


### Model Training

In [43]:
gboost.fit(x_train,y_train)
populate_matrix(gboost)
print(my_dict)

{'model': ['LogisticRegression()', 'DecisionTreeClassifier()', 'RandomForestClassifier()', 'GradientBoostingClassifier()'], 'accuracy': [86.51, 93.4, 97.45, 97.6], 'precision': [56.6, 73.45, 100.0, 97.65], 'recall': [30.93, 85.57, 82.47, 85.57]}


## SVM

In [44]:

svm = SVC()
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(estimator=svm,X=X,y=y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.928 (0.011)


### Model Training

In [45]:
svm.fit(x_train,y_train)
populate_matrix(svm)
print(my_dict)

{'model': ['LogisticRegression()', 'DecisionTreeClassifier()', 'RandomForestClassifier()', 'GradientBoostingClassifier()', 'SVC()'], 'accuracy': [86.51, 93.4, 97.45, 97.6, 92.8], 'precision': [56.6, 73.45, 100.0, 97.65, 88.89], 'recall': [30.93, 85.57, 82.47, 85.57, 57.73]}


## ANN

In [85]:
ann_model = Sequential()
ann_model.add(Dropout(0.30))
ann_model.add(Dense(20,input_dim=x_train.shape[1],activation='relu'))
ann_model.add(Dense(10,activation='relu'))
ann_model.add(Dropout(0.30))
ann_model.add(Dense(5,activation='relu'))
ann_model.add(Dense(1,activation='sigmoid'))


In [86]:
ann_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


### Model Training

In [87]:
ann_model.fit(x_train,y_train,batch_size=100,epochs=200,validation_data=(x_test,y_test),shuffle=True)



Epoch 1/200
27/27 [==============================] - 1s 6ms/step - loss: 0.7954 - accuracy: 0.3601 - val_loss: 0.6799 - val_accuracy: 0.5532
Epoch 2/200
27/27 [==============================] - 0s 2ms/step - loss: 0.6570 - accuracy: 0.6148 - val_loss: 0.5975 - val_accuracy: 0.8021
Epoch 3/200
27/27 [==============================] - 0s 2ms/step - loss: 0.5856 - accuracy: 0.7674 - val_loss: 0.5318 - val_accuracy: 0.8561
Epoch 4/200
27/27 [==============================] - 0s 2ms/step - loss: 0.5312 - accuracy: 0.8410 - val_loss: 0.4765 - val_accuracy: 0.8546
Epoch 5/200
27/27 [==============================] - 0s 2ms/step - loss: 0.4900 - accuracy: 0.8518 - val_loss: 0.4342 - val_accuracy: 0.8561
Epoch 6/200
27/27 [==============================] - 0s 2ms/step - loss: 0.4681 - accuracy: 0.8530 - val_loss: 0.4080 - val_accuracy: 0.8561
Epoch 7/200
27/27 [==============================] - 0s 2ms/step - loss: 0.4473 - accuracy: 0.8571 - val_loss: 0.3903 - val_accuracy: 0.8561
Epoch 8/200
2

Epoch 59/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2905 - accuracy: 0.9025 - val_loss: 0.2385 - val_accuracy: 0.9115
Epoch 60/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2772 - accuracy: 0.9014 - val_loss: 0.2375 - val_accuracy: 0.9130
Epoch 61/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2817 - accuracy: 0.9010 - val_loss: 0.2360 - val_accuracy: 0.9130
Epoch 62/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2902 - accuracy: 0.9010 - val_loss: 0.2390 - val_accuracy: 0.9130
Epoch 63/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2726 - accuracy: 0.9017 - val_loss: 0.2378 - val_accuracy: 0.9160
Epoch 64/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2833 - accuracy: 0.8991 - val_loss: 0.2381 - val_accuracy: 0.9130
Epoch 65/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2847 - accuracy: 0.8968 - val_loss: 0.2407 - val_accuracy: 0.9160
Epoch 

27/27 [==============================] - 0s 2ms/step - loss: 0.2702 - accuracy: 0.9059 - val_loss: 0.2290 - val_accuracy: 0.9145
Epoch 117/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2632 - accuracy: 0.9141 - val_loss: 0.2272 - val_accuracy: 0.9160
Epoch 118/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2658 - accuracy: 0.9100 - val_loss: 0.2296 - val_accuracy: 0.9115
Epoch 119/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2574 - accuracy: 0.9152 - val_loss: 0.2293 - val_accuracy: 0.9130
Epoch 120/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2709 - accuracy: 0.9085 - val_loss: 0.2252 - val_accuracy: 0.9175
Epoch 121/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2732 - accuracy: 0.9096 - val_loss: 0.2428 - val_accuracy: 0.9085
Epoch 122/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2565 - accuracy: 0.9149 - val_loss: 0.2297 - val_accuracy: 0.9115
Epoch 123/200

27/27 [==============================] - 0s 2ms/step - loss: 0.2542 - accuracy: 0.9145 - val_loss: 0.2361 - val_accuracy: 0.9070
Epoch 174/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2509 - accuracy: 0.9179 - val_loss: 0.2386 - val_accuracy: 0.9070
Epoch 175/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2559 - accuracy: 0.9182 - val_loss: 0.2341 - val_accuracy: 0.9085
Epoch 176/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2527 - accuracy: 0.9171 - val_loss: 0.2372 - val_accuracy: 0.9055
Epoch 177/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2481 - accuracy: 0.9134 - val_loss: 0.2341 - val_accuracy: 0.9055
Epoch 178/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2580 - accuracy: 0.9141 - val_loss: 0.2338 - val_accuracy: 0.9055
Epoch 179/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2447 - accuracy: 0.9186 - val_loss: 0.2320 - val_accuracy: 0.9070
Epoch 180/200

### convertiing probabilities to labels

In [88]:
y_pred = ann_model.predict(x_test)
for i in range(0,len(y_pred)):
    if y_pred[i]>=.5:
        y_pred[i] = 1
    else:
        y_pred[i] = 0
   

In [97]:
my_dict["model"].append('ANN')
my_dict["accuracy"].append(round(accuracy_score(y_test,y_pred)*100,2))
my_dict["precision"].append(round(precision_score(y_test,y_pred)*100,2))
my_dict["recall"].append(round(recall_score(y_test,y_pred)*100,2))

### Final performance matrix of all the models

In [99]:
pd.DataFrame(my_dict)

,model,accuracy,precision,recall
0,LogisticRegression(),86.51,56.60,30.93
1,DecisionTreeClassifier(),93.40,73.45,85.57
2,RandomForestClassifier(),97.45,100.00,82.47
3,GradientBoostingClassifier(),97.60,97.65,85.57
4,SVC(),92.80,88.89,57.73
5,ANN,91.00,97.44,39.18
